## Copies data from blob to local host

##### Copyright (C) Microsoft Corporation.  
see license file for details 

In [1]:
# Allow multiple displays per cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# AZUREML_NATIVE_SHARE_DIRECTORY mapping to host dir is set by _nativeSharedDirectory_ in .compute file 

import os
try:
    amlWBSharedDir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']    
except:
    amlWBSharedDir =  os.path.join(os.getcwd(), os.path.join(*(['..', '..', '..', '..'])))
    print('not using AML Workbench?')
    
amlWBSharedDir

not using AML Workbench?


'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../..'

In [4]:
# Use the Azure Machine Learning data collector to log various metrics

# from azureml.logging import get_azureml_logger
# logger = get_azureml_logger()

In [5]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on 

In [6]:
# import utlity functions

import sys
paths_to_append = [os.path.join(os.getcwd(), os.path.join(*(['..','src'])))]
paths_to_append
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import azure_chestxray_utils

['/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../src']

[None]

In [7]:
# create the file path variables 
# create nih_chest_xray_data_dir (in container dir mapped to a host dir for data persistence), 
# where data will be copied from blob

prj_consts = azure_chestxray_utils.chestxray_consts()

os.getcwd()


data_base_input_dir=os.path.join(amlWBSharedDir, os.path.join(*(prj_consts.BASE_INPUT_DIR_list)))
credential_info_path=os.path.join(os.getcwd(), os.path.join(*(['..','..','..']+prj_consts.CREDENTIALS_DIR_list)))
nih_chest_xray_data_dir=os.path.join(data_base_input_dir, 
                                     os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))

!mkdir -p {data_base_input_dir}
!mkdir -p {credential_info_path}  
!mkdir -p {nih_chest_xray_data_dir}
!chmod -R ugo=rwx  {data_base_input_dir}

data_base_input_dir
credential_info_path
nih_chest_xray_data_dir

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep'

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8'

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../notShared'

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC'

In [8]:
# Create this function in file '{credential_info_path}/get_data_access_secrets.py'
# def get_blob_credentials():
#     dataBlob = 'https://somedatastore.blob.core.windows.net/somecontainer'
#     sourceKey = 'somesourceKey'
#     return dataBlob, sourceKey
# 
# For example:
if 0:
    CredentialsFileName = os.path.join(credential_info_path, 'get_data_access_secrets.py') 
    crt_container = 'https://somedatastore.blob.core.windows.net/somecontainer'
    crt_source_Key = 'some_key'
    
    import os.path 

    if not os.path.isfile(CredentialsFileName): 
        print('Credentials file not found, will be written!') 
        with open(CredentialsFileName,'w') as myFile:
            myFile.write(("def get_blob_credentials():\n\t" +
            "dataBlob = '" + crt_container + "'\n\t" +
            "sourceKey = '" + crt_source_Key + "'\n\t" +
            "return dataBlob, sourceKey\n" )) 
        print(('Credentials file written at ' +CredentialsFileName)) 
    else: 
        print(('Credentials file found at ' + CredentialsFileName))
    
# check CredentialsFileName existence and print content if needed

credentials_file_name = os.path.join(*([credential_info_path, 'get_data_access_secrets.py']))
credentials_file_name
# !cat {credentials_file_name}

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../notShared/get_data_access_secrets.py'

In [9]:
# read credentials info

add_path_to_sys_path(credential_info_path)         
import get_data_access_secrets   
crt_container, crt_key = get_data_access_secrets.get_blob_credentials()
# crt_container
# crt_key

In [10]:
#check azcopy is available. Uncomment second line to print _response_ if needed

response = !azcopy
response[1:10]

['azcopy 7.1.0-netcorepreview Copyright (c) 2017 Microsoft Corp. All Rights Reserved.',
 '------------------------------------------------------------------------------',
 '# azcopy is designed for high-performance uploading, downloading, and copying',
 'data to and from Microsoft Azure Blob, and File storage.',
 '',
 '# Command Line Usage:',
 '    azcopy --source <source> --destination <destination> [options]',
 '',
 '# Options:']

In [11]:
# copy data to local host dir
# add {  --source-key {crt_key}  } if needed

tmp_dir1 = os.path.join(nih_chest_xray_data_dir, '..')

!azcopy --quiet \
    --source {crt_container}  \
    --destination {tmp_dir1} \
    --source-key {crt_key} \
    --verbose \
    --include "BBox_List_2017.csv"

!azcopy --quiet \
    --source {crt_container}  \
    --destination {tmp_dir1} \
    --source-key {crt_key} \
    --verbose \
    --include "Data_Entry_2017.csv"

!azcopy --quiet \
    --source {crt_container}  \
    --destination {tmp_dir1} \
    --source-key {crt_key} \
    --verbose \
    --include "blacklist.csv"

=[2018/09/26 19:58:52.110+00:00] >>>>>>>>>>>>>>>>
[2018/09/26 19:58:52.116+00:00][VERBOSE] Finished: 0 file(s), 0 B; Average Speed:0 B/s.
[2018/09/26 19:58:52.118+00:00][VERBOSE] 7.1.0 : azcopy --quiet --source https://ghiordanchestxrbait02sa.blob.core.windows.net/chestxraydata --destination /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/.. --source-key ****** --verbose --include BBox_List_2017.csv
[2018/09/26 19:58:52.187+00:00][VERBOSE] Attempt to parse address 'https://ghiordanchestxrbait02sa.blob.core.windows.net/chestxraydata' to a directory as a candidate location succeeded.
[2018/09/26 19:58:52.189+00:00][VERBOSE] Source is interpreted as a Cloud blob directory: https://ghiordanchestxrbait02sa.blob.core.windows.net/chestxraydata.
[2018/09/26 19:58:52.191+00:00][VERBOSE] Attempt to parse address '/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/..' to a directory

In [12]:
tmp_dir = os.path.join(nih_chest_xray_data_dir, 'tmp')
tmp_dir
!mkdir -p $tmp_dir

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp'

In [13]:
answer = !azcopy --quiet \
    --source {crt_container}  \
    --destination {tmp_dir} \
    --source-key {crt_key} \
    --recursive \
    --verbose \
    --include "images_"

In [14]:
answer[-7:]

['[2018/09/26 20:04:02] Transfer summary:',
 '-----------------',
 'Total files transferred: 12',
 'Transfer successfully:   12',
 'Transfer skipped:        0',
 'Transfer failed:         0',
 'Elapsed time:            00.00:04:53']

The azcopy download process above will run for several minutes. 

Make sure you have these file sizes when it finishes:  
total __44023372__  
-rw-r--r-- 1 1000 users __2008470987__ Sep 26 17:19 images_001.tar.gz  
-rw-r--r-- 1 1000 users __3952623504__ Sep 26 19:05 images_002.tar.gz  
-rw-r--r-- 1 1000 users __3929234850__ Sep 26 17:22 images_003.tar.gz  
-rw-r--r-- 1 1000 users __3838903983__ Sep 26 17:25 images_004.tar.gz  
-rw-r--r-- 1 1000 users __3935496531__ Sep 26 17:22 images_005.tar.gz  
-rw-r--r-- 1 1000 users __3986301172__ Sep 26 17:25 images_006.tar.gz  
-rw-r--r-- 1 1000 users __4016328426__ Sep 26 19:05 images_007.tar.gz  
-rw-r--r-- 1 1000 users __4018347353__ Sep 26 19:05 images_008.tar.gz  
-rw-r--r-- 1 1000 users __4111327929__ Sep 26 19:05 images_009.tar.gz  
-rw-r--r-- 1 1000 users __4181556296__ Sep 26 19:05 images_010.tar.gz  
-rw-r--r-- 1 1000 users __4187084020__ Sep 26 19:05 images_011.tar.gz  
-rw-r--r-- 1 1000 users __2914187733__ Sep 26 17:21 images_012.tar.gz  
  
If the unzipping process below does not end up with __112120__ .png images, you may need to repeat the download process for the above 12 .tar.gz files.

In [15]:
nih_chest_xray_data_dir
tmp_dir
!ls -l $tmp_dir
# !find $nih_chest_xray_data_dir -type f | wc -l

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC'

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp'

total 44023372
-rw-r--r-- 1 keras users 2008470987 Sep 26 20:00 images_001.tar.gz
-rw-r--r-- 1 keras users 3952623504 Sep 26 20:03 images_002.tar.gz
-rw-r--r-- 1 keras users 3929234850 Sep 26 20:03 images_003.tar.gz
-rw-r--r-- 1 keras users 3838903983 Sep 26 20:03 images_004.tar.gz
-rw-r--r-- 1 keras users 3935496531 Sep 26 20:03 images_005.tar.gz
-rw-r--r-- 1 keras users 3986301172 Sep 26 20:04 images_006.tar.gz
-rw-r--r-- 1 keras users 4016328426 Sep 26 20:03 images_007.tar.gz
-rw-r--r-- 1 keras users 4018347353 Sep 26 20:03 images_008.tar.gz
-rw-r--r-- 1 keras users 4111327929 Sep 26 20:04 images_009.tar.gz
-rw-r--r-- 1 keras users 4181556296 Sep 26 20:04 images_010.tar.gz
-rw-r--r-- 1 keras users 4187084020 Sep 26 20:03 images_011.tar.gz
-rw-r--r-- 1 keras users 2914187733 Sep 26 20:02 images_012.tar.gz


The unzipping process below will run for several minutes.

While it runs, you can monitor the progress by ssh-ing 
into your compute context machine and running:   
find /data/datadrive01/data/chestxray/ChestX-ray8/ChestXray-NIHCC/images/  -type f | wc -l  

The final number of images is:  
__112120__  



In [16]:
nih_chest_xray_data_dir
old_crt_dir = os.getcwd()
os.chdir(nih_chest_xray_data_dir)
!pwd
!ls -l $tmp_dir/*.tar.gz 
allImages = ! cat $tmp_dir/*.tar.gz | tar -zxf - -i
# !for file in $tmp_dir/*.tar.gz; do tar -zxf $file; done


'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC'

/local_dir/data/chestxray/ChestX-ray8/ChestXray-NIHCC
-rw-r--r-- 1 keras users 2008470987 Sep 26 20:00 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_001.tar.gz
-rw-r--r-- 1 keras users 3952623504 Sep 26 20:03 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_002.tar.gz
-rw-r--r-- 1 keras users 3929234850 Sep 26 20:03 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_003.tar.gz
-rw-r--r-- 1 keras users 3838903983 Sep 26 20:03 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_004.tar.gz
-rw-r--r-- 1 keras users 3935496531 Sep 26 20:03 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_005.tar.gz
-rw-r--r-- 1 keras users 3986301172 Sep 26 20:04 /local_

In [17]:
# os.chdir(nih_chest_xray_data_dir)
# !pwd
# !tar -zxvf $tmp_dir/images_001.tar.gz

In [18]:
os.chdir(old_crt_dir) 
!jupyter nbconvert --to html 001_get_data.ipynb

[NbConvertApp] Converting notebook 001_get_data.ipynb to html
[NbConvertApp] Writing 274792 bytes to 001_get_data.html
